In [ ]:
import advent
import numpy as np

data = advent.get_lines_doublenewline(19)

def parse_scanner(data):
    data = [[int(c) for c in line.split(',')] for line in data[1:]]
    return np.array(data)

data = list(map(parse_scanner, data))
data[0]

In [ ]:
def to_vector(l, r):
    # The vector between two beacons is defined as
    # a - b, where a[x] > b[x]
    if l[0] >= r[0]:
        return (l[0] - r[0], l[1] - r[1], l[2] - r[2])
    return (r[0] - l[0], r[1] - l[1], r[2] - l[2])

def offset(a, b, c):
    return (a[0] + b[0] - c[0], a[1] + b[1] - c[1], a[2] + b[2] - c[2])

def group_of_vectors(beacons):
    vectors = {}
    for i in range(len(beacons)):
        for j in range(i+1, len(beacons)):
            vectors[to_vector(beacons[i], beacons[j])] = (i, j)
    return vectors

# group_of_vectors(data[0])

In [ ]:
def match_old(left, right, l_vectors=None, r_vectors=None):
    # given two probes, try to match them
    # The blind assumption is (!!!):
    # if l[i1] and l[i2] have the same vector as r[j1] and r[j2],
    # Then we assume l[i1] and r[j1] are the same and l[i2] and r[j2]
    # so we will return a set containing {(l[i1], r[j1]), etc...}
    # In general, according to the exercise, we only really care
    # if we find at least 12 matches, meaning len(result) >= 12
    if l_vectors is None: l_vectors = group_of_vectors(left)
    if r_vectors is None: r_vectors = group_of_vectors(right)
    result = set([])
    for vector in l_vectors:
        if vector in r_vectors:
            l1, l2 = l_vectors[vector]
            r1, r2 = r_vectors[vector]
            result.add((l1, r1))
            result.add((l2, r2))
    return result

def match(left, right, min_match=12):
    # Better approach: 'offset' right by some amount to oversay it on left
    # then count how many beacons overlay exactly,
    # if it's 12, return the list of overlaid beacons
    # the offset is l[i] - r[j] for every i/j
    sleft = set(left)
    for i in range(len(left)):
        for j in range(len(right)):
            right_offset = [offset(b, left[i], right[j]) for b in right]
            matches = len(sleft & set(right_offset))
            if matches >= min_match:
                # found a match!
                matches = sleft & set(right_offset)
                result = []
                for i in range(len(left)):
                    if left[i] in matches:
                        for j in range(len(right)):
                            if right_offset[j] == left[i]:
                                result.append((i, j))
                return result
    return []

                    


match([(0,2,0), (4,1,0), (3,3,0)], [(-5,0,0), (-1,-1,0), (-2,1,0)], min_match=3)

In [ ]:
# The next step is the rotation (24 ways!)
# The rotations are all the even permutations (MATH!)
# 6 permutations, with 4 'signs'/flips each
rotations = [
    lambda x, y, z: (x, y, z),
    lambda x, y, z: (-x, -y, z),
    lambda x, y, z: (-x, y, -z),
    lambda x, y, z: (x, -y, -z),

    lambda x, y, z: (-x, z, y),
    lambda x, y, z: (x, -z, y),
    lambda x, y, z: (x, z, -y),
    lambda x, y, z: (-x, -z, -y),

    lambda x, y, z: (-z, y, x),
    lambda x, y, z: (z, -y, x),
    lambda x, y, z: (z, y, -x),
    lambda x, y, z: (-z, -y, -x),

    lambda x, y, z: (-y, x, z),
    lambda x, y, z: (y, -x, z),
    lambda x, y, z: (y, x, -z),
    lambda x, y, z: (-y, -x, -z),

    lambda x, y, z: (y, z, x),
    lambda x, y, z: (y, -z, -x),
    lambda x, y, z: (-y, z, -x),
    lambda x, y, z: (-y, -z, x),

    lambda x, y, z: (z, x, y),
    lambda x, y, z: (z, -x, -y),
    lambda x, y, z: (-z, x, -y),
    lambda x, y, z: (-z, -x, y)
]

def apply_rotation(beacon, rot_i):
    x, y, z = beacon
    return rotations[rot_i](x, y, z)

def apply_rotation_all(beacons, rot_i):
    return [apply_rotation(beacon, rot_i) for beacon in beacons]

def find_n_matches(left, right, min_matches=12):
    # Keep left the same, but rotate right
    # continue until we find at least 12 matches
    for rot_i in range(24):
        right_rot = apply_rotation_all(right, rot_i)
        matches = match(left, right_rot)
        if len(matches) >= min_matches:
            return matches, rot_i
    return [], -1

In [ ]:
# Just for testing

small = advent.get_lines_doublenewline('19small')
small = list(map(parse_scanner, small))

# convert to list of list of tuples
small = [[tuple(x) for x in s] for s in small]

find_n_matches(small[0], small[1])

In [ ]:
def convert_coordinates(left, right, rot_i, pair):
    # convert the coordinates of r as if scanner l is at 0,0,0
    # rot_i is the rotation that was used to make l and r match
    # pair is (i, j) such that l[i] and r[j] are the same after rotating r
    new_r = apply_rotation_all(right, rot_i)
    scanner = offset((0, 0, 0), left[pair[0]], new_r[pair[1]])
    return [offset(b, left[pair[0]], new_r[pair[1]]) for b in new_r], scanner

In [ ]:
# The goal is: we keep trying until we connect something to the 'connected_set'
# and then we add that match to matches
connected_set = set([0])
exhausted_set = set([])
matches = {}
tdata = [[tuple(x) for x in s] for s in data]

coordinates = set(tdata[0])
connected_coordinates = {0: tdata[0]}
scanner_coordinates = {0: (0, 0, 0)}

while len(connected_set) < len(data):
    for i in connected_set.copy():
        if i in exhausted_set: continue
        for j in range(len(data)):
            if j in connected_set: continue
            print(i, j)
            matches[(i, j)] = find_n_matches(connected_coordinates[i], tdata[j])
            if len(matches[(i, j)][0]) >= 12:
                connected_set.add(j)
                new_coords, scanner = convert_coordinates(
                    connected_coordinates[i],
                    tdata[j],
                    matches[(i, j)][1],
                    matches[(i, j)][0][0]
                )
                coordinates.update(new_coords)
                connected_coordinates[j] = new_coords
                scanner_coordinates[j] = scanner
        exhausted_set.add(i)
len(connected_set) # takes ~50 seconds

In [ ]:
def manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1]) + abs(a[2]-b[2])

clist = list(scanner_coordinates.values())
print(len(clist))

max_manhattan = 0
for i in range(len(clist)):
    for j in range(len(clist)):
        m = manhattan(clist[i], clist[j])
        if m > max_manhattan:
            print(m, clist[i], clist[j])
            max_manhattan = m
print(max_manhattan)

In [ ]:
# Failed approach: (before I added the 'coordinates/connected_coordinates')

In [ ]:
# [l for l in list(matches) if len(matches[l]) >= 12]

# The next step is: for each beacon (e.g. beacon 0,0 means its the first
# beacon on scanner 0's list)
# keep track of all other beacons that are the same
# e.g. if matches[(0, 5)] contains the elementn (0, 2), then we know that
# 0,0 and (5,2) are the same. if we later find out (5,2) and (10,6) are the same,
# we add that one too, etcetera, until we have a list of all unique beacons

def have_beacon(beacons, b1, b2):
    for i, beacon in enumerate(beacons):
        if b1 in beacon or b2 in beacon:
            return i
    return None

def have_beacon_one(beacons, b1):
    for i, beacon in enumerate(beacons):
        if b1 in beacon:
            return i
    return None

def add_beacon(beacons, b1, b2):
    if b2 is None:
        ix = have_beacon_one(beacons, b1)
    else:
        ix = have_beacon(beacons, b1, b2)
    if ix is not None:
        beacons[ix].add(b1)
        beacons[ix].add(b2)
    else:
        beacons.append(set([b1, b2]))
    return beacons

def add_matches(beacons, matches, l, r):
    for b in matches[(l, r)]:
        b1, b2 = (l, b[0]), (r, b[1])
        beacons = add_beacon(beacons, b1, b2)
    return beacons

beacons = []
for m in matches:
    if len(matches[m]) >= 12:
        add_matches(beacons, matches, m[0], m[1])

# Now add all the unconnected beacons:
for i in range(len(data)):
    for j in range(len(data[i])):
        if have_beacon_one(beacons, (i, j)) is not None:
            continue
        beacons = add_beacon(beacons, (i, j), None)

len(beacons) # 178 is too low :(. 411 is too high... :(

# Basically, I think the problem is two unmatched beacons
# may still be matches, just that the two scanners that detected them were too
# far apart to detect the match
# The only solution I can think of is to actually get the list of coordinates
# I hope to god this will also help during part 2 because it seems like a lot
# of effort...